<a href="https://colab.research.google.com/github/DatasMum/reactdemo/blob/main/Retrain_Object_Detection_ssd_mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction


In this notebook, we implement [The TensorFlow 2 Object Detection Library](https://blog.tensorflow.org/2020/07/tensorflow-2-meets-object-detection-api.html) for training on your own dataset.


We will take the following steps to implement mobilenetV2 on our custom data:
* Install TensorFlow2 Object Detection Dependencies
* Download Custom TensorFlow2 Object Detection Dataset
* Write Custom TensorFlow2 Object Detection Training Configuation
* Train Custom TensorFlow2 Object Detection Model
* Export Custom TensorFlow2 Object Detection Weights
* Use Trained TensorFlow2 Object Detection For Inference on Test Images

When you are done you will have a custom detector that you can use. 

# Install TensorFlow2 Object Detection Dependencies

In [1]:
#Clone rep of TensorFlow object detection api 
import os
import pathlib

%cd /content/

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

/content
Cloning into 'models'...
remote: Enumerating objects: 3157, done.
remote: Counting objects: 100% (3157/3157), done.
remote: Compressing objects: 100% (2490/2490), done.
remote: Total 3157 (delta 836), reused 1498 (delta 623), pack-reused 0
Receiving objects: 100% (3157/3157), 33.37 MiB | 18.43 MiB/s, done.
Resolving deltas: 100% (836/836), done.


In [2]:
# Install the Object Detection API
%%bash
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684828 sha256=76f89e585d3dfa3bd95ad483ff43eb5ed5ebee87d7c7f152b2c728e1ab97f209
  Stored in directory: /tmp/pip-ephem-wheel-cache-0hdt40eo/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=870aa9cabf811619c4de1bf3f27767ddec65a87ff249d1bf97be424a428474bf
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=15b5662eeb9e255e1fd354d19e0f42a94e5b9849642ab2b8816298d62e18cb5f
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=1eb382fb8cdebe2908353ad962efe9edfb4806e85bb6ab01a0

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [4]:
#run model builder test
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py


Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-02-02 06:08:17.228739: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
W0202 06:08:17.576105 140338018654080 model_builder.py:1100] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.74s
I0202 06:08:17.836381 140338018654080 test_util.py:2309] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 2.74s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.5s
I0202 06:08:18.338307 140338018654080 test_

# Prepare Tensorflow 2 Object Detection Training Data

> 



We are going to use Roboflow to generate image data set and convert it to TFrecords format.

To create a dataset in Roboflow and generate TFRecords, follow [this step-by-step guide](https://blog.roboflow.ai/getting-started-with-roboflow/).




In [15]:
#Downloading data Training set made by Roboflow
%cd /content

#Download Training set from git by cloning rep:
import os
import pathlib
# Clone the training set repository if it doesn't already exist
if "RetrainModelExample" in pathlib.Path.cwd().parts:
  while "RetrainModelExample" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('RetrainModelExample').exists():
  !git clone --depth 1 https://github.com/KostaMalsev/RetrainModelExample
  %cd /content/RetrainModelExample/TrainingSet/Picka 
  !unzip Pickachu2.v1.tfrecord.zip -d /content/

#NOTE: Update these TFRecord names to your files containing training set!
#Also, Update relevant rows:in training config file "ssd_mobilenet_v2_320x320_coco17_tpu-8.config"
#label_map_path,input_path 
test_record_fname = '/content/valid/Cars.tfrecord'
train_record_fname = '/content/train/Cars.tfrecord'
label_map_pbtxt_fname = '/content/train/Cars_label_map.pbtxt'

#test_record_fname = '/content/valid/pieces.tfrecord'
#train_record_fname = '/content/train/toymnm.tfrecord'
#label_map_pbtxt_fname = '/content/train/toymnm_label_map.pbtxt'


/content


# Configure Custom TensorFlow2 Object Detection Training 




> In this section we specify configuration for mobilentV2 model. for additional models see [TF2 OD model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).



In [12]:
#We choose mobilentv2 model to deploy from TF2 object detection zoo
MODELS_CONFIG = {
    'ssd_mobilenet_v2_320x320_coco17': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 16
    }
}
#'batchsize 512 19/10/20
chosen_model = 'ssd_mobilenet_v2_320x320_coco17'

num_steps = 1800 #40000 #The more steps, the longer the training. Increase if your loss function is still decreasing and validation metrics are increasing. 
num_eval_steps = 500 #Perform evaluation after so many steps

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your trainin#g
#base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']

In [13]:
#Download pretrained weights
%mkdir /content/deploy/
%cd /content/deploy/
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()
#Shorten the folder name,because long file paths are not yet supported :(
os.rename('ssd_mobilenet_v2_320x320_coco17_tpu-8','mobilnetv2')

mkdir: cannot create directory ‘/content/deploy/’: File exists
/content/deploy
--2022-02-02 06:14:45--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_32 100%[===================>]  43.91M   257MB/s    in 0.2s    

2022-02-02 06:14:45 (257 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz.1’ saved [46042990/46042990]



OSError: ignored

In [8]:
#Download training configuration file for mobilenetV2. 
#note: configuration file contain references to your trainig set of images,
#you can change it for your dataset.
%cd /content/deploy
download_config = 'https://raw.githubusercontent.com/KostaMalsev/RetrainModelExample/main/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
!wget {download_config}

/content/deploy
--2022-02-02 06:10:21--  https://raw.githubusercontent.com/KostaMalsev/RetrainModelExample/main/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4737 (4.6K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_32 100%[===================>]   4.63K  --.-KB/s    in 0s      

2022-02-02 06:10:22 (63.3 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’ saved [4737/4737]



In [20]:
#Prepare loaded model for retraining
fine_tune_checkpoint = '/content/deploy/mobilnetv2/checkpoint/ckpt-0'
pipeline_file = '/content/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
model_dir = '/content/training/'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)


In [21]:
#Check if all configuration is OK:
print(fine_tune_checkpoint)
print(train_record_fname)
print(label_map_pbtxt_fname)
print(batch_size)
print(num_steps)
print(num_classes)
print(pipeline_file)
print(model_dir)

/content/deploy/mobilnetv2/checkpoint/ckpt-0
/content/train/Cars.tfrecord
/content/train/Cars_label_map.pbtxt
16
1800
2
/content/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
/content/training/


# Train Custom TF2 Object Detector

* pipeline_file: defined above in writing custom training configuration
* model_dir: the location tensorboard logs and saved model checkpoints will save to
* num_train_steps: how long to train for
* num_eval_steps: perform eval on validation set after this many steps







In [24]:
!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2022-02-02 06:18:31.468309: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0202 06:18:31.471405 139685939885952 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 1800
I0202 06:18:31.475993 139685939885952 config_util.py:552] Maybe overwriting train_steps: 1800
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0202 06:18:31.476160 139685939885952 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0202 06:18:31.520947 139685939885952 deprecation.py:347] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experiment

In [25]:
!pip install opencv-python-headless==4.1.2.30 

In [26]:
#Your trained weights will be in this directory:
%ls -l '/content/training/'

total 54556
-rw-r--r-- 1 root root      255 Feb  2 06:23 checkpoint
-rw-r--r-- 1 root root 18649306 Feb  2 06:19 ckpt-1.data-00000-of-00001
-rw-r--r-- 1 root root    22263 Feb  2 06:19 ckpt-1.index
-rw-r--r-- 1 root root 37130982 Feb  2 06:23 ckpt-2.data-00000-of-00001
-rw-r--r-- 1 root root    41640 Feb  2 06:23 ckpt-2.index
drwxr-xr-x 2 root root     4096 Feb  2 06:18 train/


In [27]:
#Run conversion script to save the retrained model:
#Saved model will be in saved_model.pb file:

import re
import numpy as np

output_directory = '/content/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/content/training/'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/content/training/
2022-02-02 06:27:55.300221: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
W0202 06:27:55.797803 140577799800704 deprecation.py:619] From /usr/local/lib/python3.7/dist-packages/tensorflow/python/autograph/impl/api.py:464: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
INFO:tensorflo

In [28]:
%ls '/content/fine_tuned_model/saved_model/'

assets/  saved_model.pb  variables/


# Run Inference on Test Images with Custom TensorFlow2 Object Detector

In [29]:
#Import your test images to colab. I use pinterest to store the the images. 
%mkdir /content/test/
%cd /content/test/
#M&M toy:
#!curl -L "https://i.pinimg.com/originals/4c/6a/00/4c6a0021a735e1dcb9edcb6715467e15.jpg" > test.jpeg;
#Pickachu toy:
!curl -L "https://i.pinimg.com/564x/f5/46/c4/f546c47505e1f5f8d17f8458d641b262.jpg" > test.jpeg;


/content/test
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 66488  100 66488    0     0   532k      0 --:--:-- --:--:-- --:--:--  532k


In [30]:
import os 
import glob
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [31]:
#Recover our saved model with the latest checkpoint:
pipeline_config = pipeline_file
#Put the last ckpt from training in here, don't use long pathnames:
model_dir = '/content/training/ckpt-2'
configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
detection_model = model_builder.build(
      model_config=model_config, is_training=False)

# Restore last checkpoint
ckpt = tf.compat.v2.train.Checkpoint(
      model=detection_model)
#ckpt.restore(os.path.join(model_dir))
ckpt.restore(model_dir)

#Function perform detection of the object on image in tensor format: 
def get_model_detection_function(model):
  """Get a tf.function for detection."""

  @tf.function
  def detect_fn(image):
    """Detect objects in image."""
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)

    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn
  
#Define function which performs detection: 
detect_fn = get_model_detection_function(detection_model)

In [32]:
#map labels for inference decoding
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

In [61]:
#run detector on test image
#it takes a little longer on the first run and then runs at normal speed. 
import random

#Define utility functions for presenting the results:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.
  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.
  Args:
    path: the file path to the image
  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)


#Place your test images here:
image_path = '/content/test/overlay.jpg'

#Store test images in nmpy array:
image_np = load_image_into_numpy_array(image_path)

#Convert images to tensor form:
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)

#Perform detection on the image in tensor format:
detections, predictions_dict, shapes = detect_fn(input_tensor)

#Visualize the detection boxes on the image:
label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=0.70,#0.5,#0.5
      agnostic_mode=False,
)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()

TypeError: ignored

In [60]:
#Place your test images here:
image_path = '/content/test/test.jpeg'

#Store test images in nmpy array:
image_np = load_image_into_numpy_array(image_path)

#Convert images to tensor form:
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)

#Perform detection on the image in tensor format:
detections, predictions_dict, shapes = detect_fn(input_tensor)


TypeError: ignored

In [51]:
img_data = tf.io.gfile.GFile('/content/test/overlay.jpg', 'rb').read()
image = Image.open(BytesIO(img_data))
(im_width, im_height) = image.size
np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

array([[[115, 114,  83],
        [105, 104,  73],
        [ 87,  86,  55],
        ...,
        [255, 220, 165],
        [255, 217, 162],
        [254, 215, 160]],

       [[ 94,  93,  62],
        [ 85,  84,  53],
        [ 82,  81,  50],
        ...,
        [238, 222, 189],
        [237, 221, 188],
        [235, 219, 186]],

       [[ 80,  79,  48],
        [ 77,  76,  45],
        [ 89,  88,  57],
        ...,
        [ 28,  33,  29],
        [ 28,  33,  29],
        [ 27,  32,  28]],

       ...,

       [[ 28,  52,  20],
        [ 50,  74,  42],
        [ 47,  71,  39],
        ...,
        [ 91, 114,  58],
        [ 66,  89,  35],
        [ 56,  79,  27]],

       [[ 43,  67,  35],
        [ 59,  83,  51],
        [ 48,  72,  40],
        ...,
        [108, 131,  77],
        [ 86, 109,  57],
        [ 73,  95,  46]],

       [[ 40,  64,  32],
        [ 54,  78,  46],
        [ 41,  65,  31],
        ...,
        [121, 141,  88],
        [110, 130,  79],
        [ 98, 118,  69]]

In [ ]:
print(detections)

{'detection_boxes': <tf.Tensor: shape=(1, 100, 4), dtype=float32, numpy=
array([[[0.        , 0.5384983 , 0.8673439 , 1.        ],
        [0.06221145, 0.0056223 , 0.8854016 , 0.25180617],
        [0.38846886, 0.231343  , 0.79521966, 0.4667509 ],
        [0.18573296, 0.59499484, 1.        , 1.        ],
        [0.4002268 , 0.        , 0.998803  , 0.35892016],
        [0.00523809, 0.        , 0.7317513 , 0.39898866],
        [0.        , 0.44331908, 0.45566368, 1.        ],
        [0.5748647 , 0.5584329 , 1.        , 0.9606729 ],
        [0.        , 0.21766928, 0.6137232 , 1.        ],
        [0.44021353, 0.1027469 , 1.        , 1.        ],
        [0.45164448, 0.1844042 , 0.75729114, 0.5074981 ],
        [0.44310138, 0.27960944, 0.96253586, 0.700312  ],
        [0.05797669, 0.13167095, 0.7140225 , 0.83621466],
        [0.37078822, 0.182004  , 0.6741077 , 0.52983606],
        [0.36808866, 0.        , 0.9767062 , 0.13438487],
        [0.        , 0.642269  , 0.6422963 , 0.9904635 ],

# Congrats!
Hope you enjoyed this!

# Bonus level:

To use this retrained model in web browser run the following commands:

For details see my article on medium: 

"[Build custom object detection web application using TensorFlow.js](https://kostya-malsev.medium.com/build-custom-object-detection-web-application-using-tensorflow-js-d1664f96a18b)"


In [ ]:
!saved_model_cli show --dir /content/fine_tuned_model/saved_model/ --tag_set serve --signature_def serving_default


In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter  \
--input_format=tf_saved_model\
 --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores'\
  --saved_model_tags=serve\
   --output_format=tfjs_graph_model /content/fine_tuned_model/saved_model /content/fine_tuned_model/web_model/

In [ ]:
!ls /content/fine_tuned_model/

checkpoint  pipeline.config  saved_model  web_model
